In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
cd "/content/drive/MyDrive/ClassAug Work/mixup-cifar10-master"

/content/drive/.shortcut-targets-by-id/12PFZZk88jXqRuPPPvsCgycke0gd7aQBI/ClassAug Work/mixup-cifar10-master


In [40]:
from __future__ import print_function

import argparse
import csv
import os

import numpy as np
import torch
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn.model_selection import train_test_split
import pickle
import models

In [41]:
lr=0.1
resume=False
model='ResNet50'
name='0'
seed=0
batch_size=128
epoch=1000
augment=True
decay=0.0001
alpha=1.0

In [42]:
class IDDataset(Dataset):
  def __init__(self, embs, labels):
    self.embs = torch.from_numpy(embs).float()
    self.labels = torch.from_numpy(np.squeeze(labels)).to(dtype=torch.long)

  def __len__(self):
    return self.embs.shape[0]

  def __getitem__(self, idx):
    return self.embs[idx], self.labels[idx]

class OODDataset(Dataset):
  def __init__(self, embs):
    self.embs = torch.from_numpy(embs).float()

  def __len__(self):
    return self.embs.shape[0]

  def __getitem__(self, idx):
    return self.embs[idx], 0

In [43]:
class OEModel(nn.Module):
    def __init__(self, num_classes):
        super(OEModel, self).__init__()

        self.l1 = nn.Linear(2048, 1024)
        self.classifier = nn.Linear(1024, num_classes)
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.l1(x)
        x = self.classifier(x)
        x = self.softmax(x)
        return x

In [44]:
use_cuda = torch.cuda.is_available()

best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

if seed != 0:
    torch.manual_seed(seed)

# Data
print('==> Preparing data..')
if augment:
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010)),
    ])
else:
    transform_train = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010)),
    ])


transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
embs_data = None
with open('../embeddings/svhn_resnet50_embeddings.pkl', 'rb') as f:
    embs_data = pickle.load(f)
embs = embs_data['embs']
labels = embs_data['labels']
x_train, x_test, y_train, y_test = train_test_split(embs, labels, test_size=0.30, random_state=43)
trainset = IDDataset(x_train, y_train)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=batch_size,
                                          shuffle=True, num_workers=1)

testset = IDDataset(x_test, y_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=1)


# Model
if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint_new/ckpt_resnet50.t7' + name + '_'
                            + str(seed))
    net = checkpoint['net']
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch'] + 1
    rng_state = checkpoint['rng_state']
    torch.set_rng_state(rng_state)
else:
    print('==> Building model..')
    net = OEModel(10)

==> Preparing data..
==> Building model..


In [45]:
if not os.path.isdir('results_new'):
    os.mkdir('results_new')

logname = ('results_new/log_' + net.__class__.__name__ + '_' + name + '_'
           + str(seed) + '.csv')

if use_cuda:
    net.cuda()
    net = torch.nn.DataParallel(net)
    print(torch.cuda.device_count())
    cudnn.benchmark = True
    print('Using CUDA..')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=decay)
# optimizer = optim.SGD(net.parameters(), lr=lr)

print(optimizer)

1
Using CUDA..
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0001
)


In [46]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    reg_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()

        inputs, targets_a, targets_b, lam = mixup_data(inputs, targets,
                                                       alpha, use_cuda)
        inputs, targets_a, targets_b = map(Variable, (inputs,
                                                      targets_a, targets_b))
        outputs = net(inputs)
        loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
        train_loss += loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (lam * predicted.eq(targets_a.data).cpu().sum().float()
                    + (1 - lam) * predicted.eq(targets_b.data).cpu().sum().float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return (train_loss/batch_idx, reg_loss/batch_idx, 100.*correct/total)


def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(testloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = Variable(inputs, volatile=True), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)

        test_loss += loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    acc = 100.*correct/total
    if epoch == start_epoch + epoch - 1 or acc > best_acc:
        checkpoint(acc, epoch)
    if acc > best_acc:
        print(acc)
        best_acc = acc
    return (test_loss/batch_idx, 100.*correct/total)


def checkpoint(acc, epoch):
    # Save checkpoint.
    print('Saving..')
    state = {
        'net': net,
        'acc': acc,
        'epoch': epoch,
        'rng_state': torch.get_rng_state()
    }
    if not os.path.isdir('checkpoint_new'):
        os.mkdir('checkpoint_new')
    torch.save(state, './checkpoint_new/mixup_svhn.t7' + name + '_'
               + str(seed))


def adjust_learning_rate(optimizer, epoch):
    """decrease the learning rate at 100 and 150 epoch"""
    global lr
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


if not os.path.exists(logname):
    with open(logname, 'w') as logfile:
        logwriter = csv.writer(logfile, delimiter=',')
        logwriter.writerow(['epoch', 'train loss', 'reg loss', 'train acc',
                            'test loss', 'test acc'])

print('Starting to train')
for epc in range(start_epoch, epoch):
    print(epc)
    train_loss, reg_loss, train_acc = train(epc)
    test_loss, test_acc = test(epc)
    adjust_learning_rate(optimizer, epc)
    with open(logname, 'a') as logfile:
        logwriter = csv.writer(logfile, delimiter=',')
        logwriter.writerow([epc, train_loss, reg_loss, train_acc, test_loss,
                            test_acc])

Starting to train
0

Epoch: 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Saving..
tensor(19.3776)
1

Epoch: 1
2

Epoch: 2
3

Epoch: 3
4

Epoch: 4
6

Epoch: 6
Saving..
tensor(42.2936)
7

Epoch: 7
Saving..
tensor(48.6957)
8

Epoch: 8
Saving..
tensor(50.3911)
9

Epoch: 9
10

Epoch: 10
11

Epoch: 11
Saving..
tensor(55.5477)
12

Epoch: 12
13

Epoch: 13
14

Epoch: 14
15

Epoch: 15
16

Epoch: 16
17

Epoch: 17
18

Epoch: 18
Saving..
tensor(57.3237)
19

Epoch: 19
20

Epoch: 20
21

Epoch: 21
22

Epoch: 22
23

Epoch: 23
24

Epoch: 24
Saving..
tensor(58.8411)
25

Epoch: 25
26

Epoch: 26
27

Epoch: 27
28

Epoch: 28
29

Epoch: 29
30

Epoch: 30
31

Epoch: 31
32

Epoch: 32
33

Epoch: 33
34

Epoch: 34
35

Epoch: 35
36

Epoch: 36
37

Epoch: 37
38

Epoch: 38
39

Epoch: 39
40

Epoch: 40
41

Epoch: 41
42

Epoch: 42
43

Epoch: 43
44

Epoch: 44
45

Epoch: 45
46

Epoch: 46
47

Epoch: 47
48

Epoch: 48
49

Epoch: 49
50

Epoch: 50
51

Epoch: 51
52

Epoch: 52
53

Epoch: 53
54

Epoch: 54
55

Epoch: 55
56

Epoch: 56
57

Epoch: 57
58

Epoch: 58
59

Epoch: 59
60

Epoch: 60
61

Epoch: 61
6